In [1]:
import pandas as pd
import os
import requests
import boto3
from utils import *
import datetime

**EXTRACTION**

GET SUMMONER PUUID BY SUMMONER NAME AND TAGLINE

In [2]:
api_key = os.environ.get("ETL-LOL_API")

summ_name = "TATIAN"
tag_line = "LAS"

params = {
    'api_key': api_key
}

servers = {
    'AMERICAS': 'americas.api.riotgames.com',
    'ASIA': 'asia.api.riotgames.com',
    'EUROPE': 'europe.api.riotgames.com',   
    'SEA': 'sea.api.riotgames.com'
}

endpoint = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{summ_name}/{tag_line}"

res = requests.get(endpoint, params= params)
data = res.json()

puuid = data['puuid']

df_summoner = pd.json_normalize(data)

df_summoner

,puuid,gameName,tagLine
0,rybLSc_KZF6-IQ8HCLMML90_kvd00YLW5DE0ruoXGgZF2P...,TATIAN,LAS


GET SUMMONER DATA BY PUUID

In [318]:
endpoint = f"https://la2.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"

res = requests.get(endpoint, params=params)
data = dict(res.json())

summ_id = data['id']

data.pop('puuid')

df_summoner = pd.concat([df_summoner, pd.json_normalize(data)], axis=1)

df_summoner = df_summoner.drop(columns=['profileIconId'])

df_summoner

,puuid,gameName,tagLine,id,accountId,revisionDate,summonerLevel
0,rybLSc_KZF6-IQ8HCLMML90_kvd00YLW5DE0ruoXGgZF2P...,TATIAN,LAS,lJDYoZSk-rtpKy1UnGV65cDlknAvnLOq8uvnGRFFCjIwHw,3SsoFKJ2HArAyEROZF7-cn0zHxd1m-rxRYxt9MScpr1oaME,1731213102000,362


GET CHAMPIONS LIST

In [319]:
champions_endpoint = "https://ddragon.leagueoflegends.com/cdn/14.22.1/data/en_US/champion.json"

data = requests.get(champions_endpoint).json()

df_champions = pd.DataFrame(data['data'])

df_champions = df_champions.T

df_champions.head(1)

,version,id,key,name,title,blurb,info,image,tags,partype,stats
Aatrox,14.22.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'full': 'Aatrox.png', 'sprite': 'champion0.pn...",[Fighter],Blood Well,"{'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpper..."


GET MASTERY CHAMPIONS BY SUMMONER PUUID

In [320]:
endpoint = f"https://la2.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}"

res = requests.get(endpoint, params=params)
data = res.json()

df_champions_mastery = pd.json_normalize(data)

df_champions_mastery.head(1)


,puuid,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,markRequiredForNextLevel,tokensEarned,championSeasonMilestone,nextSeasonMilestone.requireGradeCounts.B-,nextSeasonMilestone.requireGradeCounts.C-,nextSeasonMilestone.rewardMarks,nextSeasonMilestone.bonus,nextSeasonMilestone.rewardConfig.rewardValue,nextSeasonMilestone.rewardConfig.rewardType,nextSeasonMilestone.rewardConfig.maximumReward,nextSeasonMilestone.totalGamesRequires,milestoneGrades,nextSeasonMilestone.requireGradeCounts.A-
0,rybLSc_KZF6-IQ8HCLMML90_kvd00YLW5DE0ruoXGgZF2P...,64,32,372982,1717130755000,55382,-44382,2,0,0,1.0,4,1,False,5f4333db-e90d-4705-903b-08dbf5e61006,HEXTECH_CHEST,6.0,5,NaN,NaN


GET THE LAST 20 MATCHES:
1- FIRST I GET THE MATCHES IDs.
2- SINCE FROM THOSE IDs, I CAN GET MATCHES DATA 

In [ ]:
endpoint = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"

res = requests.get(endpoint, params=params)
matches_id = res.json()

matches_id

matches = []
for matchId in matches_id:
    endpoint = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}"
    
    res = requests.get(endpoint, params=params)
    matches.append(res.json())
    
df_matches = pd.json_normalize(matches)

In [326]:
df_matches.head(1)

,metadata.dataVersion,metadata.matchId,info.endOfGameResult,info.gameCreation,info.gameDuration,info.gameEndTimestamp,info.gameId,info.gameMode,info.gameName,info.gameStartTimestamp,info.gameType,info.gameVersion,info.mapId,info.platformId,info.queueId,info.tournamentCode
0,2,LA2_1460246484,GameComplete,1731212021831,971,1731213098902,1460246484,CLASSIC,teambuilder-match-1460246484,1731212127539,MATCHED_GAME,14.22.633.1362,11,LA2,420,


MATCHES DETAILS

In [324]:
df_matches_details = df_matches[["metadata.matchId", "metadata.participants","info.participants","info.teams"]]

df_matches = df_matches.drop(columns=["metadata.participants","info.participants","info.teams"])

In [325]:
df_matches_details.head(1)

,metadata.matchId,metadata.participants,info.participants,info.teams
0,LA2_1460246484,[lmY9N2GCcOkNkiEn2FCEcUSbMx0njx137z_JYJ70BjGq5...,"[{'allInPings': 0, 'assistMePings': 1, 'assist...","[{'bans': [{'championId': 910, 'pickTurn': 1},..."


GET SUMMONER RANKING BY SUMMONER ID

In [327]:
endpoint = f"https://la2.api.riotgames.com/lol/league/v4/entries/by-summoner/{summ_id}"

res = requests.get(endpoint, params=params)

data = res.json()

df_ranking = pd.json_normalize(data)

df_ranking

,leagueId,queueType,tier,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,6d220f29-c66d-4999-8bee-fc750c55fb2e,RANKED_SOLO_5x5,PLATINUM,III,lJDYoZSk-rtpKy1UnGV65cDlknAvnLOq8uvnGRFFCjIwHw,60,6,3,False,False,False,False
1,67df7375-00af-4a4f-9282-761ec6d3ca50,RANKED_FLEX_SR,EMERALD,IV,lJDYoZSk-rtpKy1UnGV65cDlknAvnLOq8uvnGRFFCjIwHw,71,4,7,False,False,False,False


GET ITEMS LIST

In [328]:
endpoint = 'https://ddragon.leagueoflegends.com/cdn/14.23.1/data/en_US/item.json'

res = requests.get(endpoint)
data = res.json()

item_list=[]
for id, data in data['data'].items():
    item={
        'id':id,
        'name':data['name'],
        'description':data['description'],
        'colloq':data['colloq'],
        'plaintext':data['plaintext'],
        'into':data.get('into'),
        'image':data['image'],
        'gold':data['gold'],
        'tags':data['tags'],
        'maps':data['maps'],
        'stats':data['stats']
    }
    item_list.append(item)
    
df_items = pd.DataFrame(item_list)

In [329]:
df_items.head(1)

,id,name,description,colloq,plaintext,into,image,gold,tags,maps,stats
0,1001,Boots,<mainText><stats><attention>25</attention> Mov...,;,Slightly increases Move Speed,"[3005, 3047, 3006, 3009, 3010, 3020, 3111, 311...","{'full': '1001.png', 'sprite': 'item0.png', 'g...","{'base': 300, 'purchasable': True, 'total': 30...",[Boots],"{'11': True, '12': True, '21': True, '22': Fal...",{'FlatMovementSpeedMod': 25}


**LOAD RAW DATA TO DATA LAKE (S3)**

CREATING S3 CLIENT

In [330]:
s3_client = boto3.client('s3', region_name='us-east-1')

CREATING BUCKET

In [331]:
#CREATING A BUCKET
s3_client.create_bucket(ACL='private', Bucket='mylolapibucket1')

{'ResponseMetadata': {'RequestId': 'BK2WSJ5KJHQ2K617',
  'HostId': 'WSfqvczvWPd/2Bqm1DquaHaplSdPyW6ISdcee3Mq2CEcJ6xSU3sDUcB6zHL4JqPMd/ZmrfFYLfg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'WSfqvczvWPd/2Bqm1DquaHaplSdPyW6ISdcee3Mq2CEcJ6xSU3sDUcB6zHL4JqPMd/ZmrfFYLfg=',
   'x-amz-request-id': 'BK2WSJ5KJHQ2K617',
   'date': 'Wed, 27 Nov 2024 18:18:26 GMT',
   'location': '/mylolapibucket1',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/mylolapibucket1'}

LOADING DATA

In [332]:
#PATH
s3_path = f"lol_data/{datetime.date.today().strftime('%Y-%m-%d')}"

upload_dataframe_to_s3(s3_client, df_summoner, f"{s3_path}/summoner.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_ranking, f"{s3_path}/ranking.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_champions, f"{s3_path}/champions.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_champions_mastery, f"{s3_path}/champions_mastery.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_matches, f"{s3_path}/matches.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_matches_details, f"{s3_path}/matches_details.parquet", "mylolapibucket1")
upload_dataframe_to_s3(s3_client, df_items, f"{s3_path}/items.parquet", "mylolapibucket1")

**TRANSFORMATION**

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, from_unixtime, explode, concat_ws, round, regexp_replace, when, monotonically_increasing_id
import findspark

CREATING SPARK SESSION

In [5]:
findspark.init()
spark_session = SparkSession.builder \
                                    .appName("LOL_API_MODERN_DATAWAREHOUSE") \
                                    .config("spark.driver.memory", "12g") \
                                    .getOrCreate()

GETTING PARQUETS FROM DATA LAKE (S3)

In [334]:
df_summoner_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/summoner.parquet"))
df_ranking_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/ranking.parquet"))
df_champions_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/champions.parquet"))
df_champions_mastery_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/champions_mastery.parquet"))
df_matches_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/matches.parquet"))
df_matches_details_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/matches_details.parquet"))
df_items_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/items.parquet"))

SUMMONER TRANSFORM

In [64]:
#JOIN SUMMONER DF AND RANKING DF
df_summoner_spark = df_summoner_spark.join(df_ranking_spark, on=df_summoner_spark["id"] == df_ranking_spark["summonerId"])
df_summoner_spark = df_summoner_spark.drop("id")

#CONVERT UNIX TIMESTAMP TO DATE
df_summoner_spark = df_summoner_spark.withColumn("revisionDate", from_unixtime(col("revisionDate") / 1000).cast("timestamp"))

columns = df_summoner_spark.columns

summoner_puuid = columns.index("puuid")
summoner_summonerId = columns.index("summonerId")

columns[summoner_puuid], columns[summoner_summonerId] = columns[summoner_summonerId], columns[summoner_puuid]

df_summoner_spark = df_summoner_spark.select(columns)

#ORDERING
df_summoner_spark = df_summoner_spark.select("puuid","summonerId","accountId","leagueId","gameName","tagLine",
                                             "summonerLevel","queueType","tier","rank","leaguePoints","wins",
                                             "losses","veteran","inactive","freshBlood","hotStreak","revisionDate")

In [65]:
df_summoner_spark.show()

+--------------------+--------------------+--------------------+--------------------+--------+-------+-------------+---------------+--------+----+------------+----+------+-------+--------+----------+---------+-------------------+
|               puuid|          summonerId|           accountId|            leagueId|gameName|tagLine|summonerLevel|      queueType|    tier|rank|leaguePoints|wins|losses|veteran|inactive|freshBlood|hotStreak|       revisionDate|
+--------------------+--------------------+--------------------+--------------------+--------+-------+-------------+---------------+--------+----+------------+----+------+-------+--------+----------+---------+-------------------+
|rybLSc_KZF6-IQ8HC...|lJDYoZSk-rtpKy1Un...|3SsoFKJ2HArAyEROZ...|6d220f29-c66d-499...|  TATIAN|    LAS|          362|RANKED_SOLO_5x5|PLATINUM| III|          60|   6|     3|  false|   false|     false|    false|2024-11-10 01:31:42|
|rybLSc_KZF6-IQ8HC...|lJDYoZSk-rtpKy1Un...|3SsoFKJ2HArAyEROZ...|67df7375-00af-4a

CHAMPIONS TRANSFORM

In [66]:
#LIST TO STRING
df_champions_spark = df_champions_spark.withColumn("tags", concat_ws(", ", col("tags")))

#EXPLODE INFO FIELD
df_champions_spark = df_champions_spark.select("*",
                                                 col("info.attack").alias("attack"),
                                                 col("info.defense").alias("defense"),
                                                 col("info.magic").alias("magic"),
                                                 col("info.difficulty").alias("difficulty"),
                                                 col("stats.hp").alias("hp"),
                                                 col("stats.hpperlevel").alias("hpPerLevel"),
                                                 col("stats.mp").alias("mp"),
                                                 col("stats.mpperlevel").alias("mpPerLevel"),
                                                 col("stats.movespeed").alias("moveSpeed"),
                                                 col("stats.armor").alias("armor"),
                                                 col("stats.armorperlevel").alias("armorPerLevel"),
                                                 col("stats.spellblock").alias("spellBlock"),
                                                 col("stats.spellblockperlevel").alias("spellBlockPerLevel"),
                                                 col("stats.attackrange").alias("attackRange"),
                                                 col("stats.hpregen").alias("hpRegen"),
                                                 col("stats.hpregenperlevel").alias("hpRegenPerLevel"),
                                                 col("stats.mpregen").alias("mpRegen"),
                                                 col("stats.mpregenperlevel").alias("mpRegenPerLevel"),
                                                 col("stats.crit").alias("crit"),
                                                 col("stats.critperlevel").alias("critPerLevel"),
                                                 col("stats.attackdamage").alias("attackDamage"),
                                                 col("stats.attackdamageperlevel").alias("attackDamagePerLevel"),
                                                 col("stats.attackspeedperlevel").alias("attackSpeedPerLevel"),
                                                 col("stats.attackspeed").alias("attackSpeed"))

df_champions_spark = df_champions_spark.drop("info", "image", "stats", "id")

df_champions_spark = df_champions_spark.withColumnRenamed("key", "championId")

df_champions_spark = df_champions_spark.withColumn("partype", when(col("partype") == "", "None").otherwise(col("partype")))

df_champions_spark = df_champions_spark.withColumn("championId", col("championId").cast("long"))

In [67]:
df_champions_spark.show(1,truncate=False)

+-------+----------+------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+----------+------+-------+-----+----------+---+----------+---+----------+---------+-----+-------------+----------+------------------+-----------+-------+---------------+-------+---------------+----+------------+------------+--------------------+-------------------+-----------+
|version|championId|name  |title           |blurb                                                                                                                                                                                                                                                          |tags   |partype   |attack|defense|magic|difficulty|hp |hpPerLevel|mp |mpPerLevel|moveSpeed|armor|armorPerLevel|spe

CHAMPIONS MASTERY TRANSFORM

In [68]:
df_champions_mastery_spark = df_champions_mastery_spark.withColumn("lastPlayTime", from_unixtime(col("lastPlayTime") / 1000).cast("timestamp"))

df_champions_mastery_spark = df_champions_mastery_spark.drop("championPointsSinceLastLevel",
                                                             "championPointsUntilNextLevel",
                                                             "markRequiredForNextLevel",
                                                             "tokensEarned",
                                                             "championSeasonMilestone",
                                                             "nextSeasonMilestone.requireGradeCounts.B-",
                                                             "nextSeasonMilestone.requireGradeCounts.C-",
                                                             "nextSeasonMilestone.rewardMarks",
                                                             "nextSeasonMilestone.bonus",
                                                             "nextSeasonMilestone.rewardConfig.rewardValue",
                                                             "nextSeasonMilestone.rewardConfig.rewardType",
                                                             "nextSeasonMilestone.rewardConfig.maximumReward",
                                                             "nextSeasonMilestone.totalGamesRequires",
                                                             "milestoneGrades",
                                                             "nextSeasonMilestone.requireGradeCounts.A-")

In [69]:
df_champions_mastery_spark.show(1, truncate=False)

+------------------------------------------------------------------------------+----------+-------------+--------------+-------------------+
|puuid                                                                         |championId|championLevel|championPoints|lastPlayTime       |
+------------------------------------------------------------------------------+----------+-------------+--------------+-------------------+
|rybLSc_KZF6-IQ8HCLMML90_kvd00YLW5DE0ruoXGgZF2PZHM8e-C6NFI9ejEKb-p2aKQkB9p6JJ0w|64        |32           |372982        |2024-05-31 01:45:55|
+------------------------------------------------------------------------------+----------+-------------+--------------+-------------------+
only showing top 1 row



MATCHES TRANSFORM

In [70]:
df_matches_spark = df_matches_spark.withColumnsRenamed({"metadata.dataVersion":"dataVersion",
                                                        "metadata.matchId":"matchId",
                                                        "info.endOfGameResult":"endOfGameResult",
                                                        "info.gameCreation":"gameCreation",
                                                        "info.gameEndTimestamp":"gameEndTimestamp",
                                                        "info.gameDuration":"gameDuration(mins)",
                                                        "info.gameId":"gameId",
                                                        "info.gameMode":"gameMode",
                                                        "info.gameName":"gameName",
                                                        "info.gameStartTimestamp":"gameStartTimestamp",
                                                        "info.gameType":"gameType",
                                                        "info.gameVersion":"gameVersion",
                                                        "info.mapId":"mapId",
                                                        "info.platformId":"platformId",
                                                        "info.queueId":"queueId",
                                                        "info.tournamentCode":"tournamentCode",
                                                        })

df_matches_spark = df_matches_spark.withColumn("gameCreation", from_unixtime(col("gameCreation") / 1000).cast("timestamp"))
df_matches_spark = df_matches_spark.withColumn("gameEndTimestamp", from_unixtime(col("gameEndTimestamp") / 1000).cast("timestamp"))
df_matches_spark = df_matches_spark.withColumn("gameStartTimestamp", from_unixtime(col("gameStartTimestamp") / 1000).cast("timestamp"))
df_matches_spark = df_matches_spark.withColumn("gameDuration(mins)", round((col("gameDuration(mins)") / 60).cast("double"), 2))

df_matches_spark = df_matches_spark.drop("tournamentCode")

In [71]:
df_matches_spark.show(1, truncate=False)

+-----------+--------------+---------------+-------------------+------------------+-------------------+----------+--------+----------------------------+-------------------+------------+--------------+-----+----------+-------+
|dataVersion|matchId       |endOfGameResult|gameCreation       |gameDuration(mins)|gameEndTimestamp   |gameId    |gameMode|gameName                    |gameStartTimestamp |gameType    |gameVersion   |mapId|platformId|queueId|
+-----------+--------------+---------------+-------------------+------------------+-------------------+----------+--------+----------------------------+-------------------+------------+--------------+-----+----------+-------+
|2          |LA2_1460246484|GameComplete   |2024-11-10 01:13:41|16.18             |2024-11-10 01:31:38|1460246484|CLASSIC |teambuilder-match-1460246484|2024-11-10 01:15:27|MATCHED_GAME|14.22.633.1362|11   |LA2       |420    |
+-----------+--------------+---------------+-------------------+------------------+-------------

MATCHES DETAILS TRANSFORM

In [60]:
df_matches_details_spark = df_matches_details_spark.withColumnsRenamed({"metadata.matchId":"matchId",
                                                                        "metadata.participants":"participant_id",
                                                                        "info.participants":"participant_info",
                                                                        "info.teams":"infoTeams"})

#METADATA PARTICIPANTS EXPLODE
participants = df_matches_details_spark.select("matchId", "participant_id")
participants = participants.withColumn("participant_id", explode("participant_id"))

#PARTICIPANTS INFO EXPLODE
participants_info = df_matches_details_spark.select("participant_info")
participants_info = participants_info.withColumn("participant_info", explode("participant_info"))
participants_info = participants_info.select("*", col("participant_info.puuid").alias("puuid"),
                                                  col("participant_info.summonerId").alias("summonerId"),
                                                  col("participant_info.teamId").alias("teamId"),
                                                  col("participant_info.summonerName").alias("summonerName"),
                                                  col("participant_info.summonerLevel").alias("summonerLevel"),
                                                  col("participant_info.win").alias("win"),
                                                  col("participant_info.lane").alias("lane"),
                                                  col("participant_info.role").alias("role"),
                                                  col("participant_info.teamPosition").alias("teamPosition"),
                                                  col("participant_info.individualPosition").alias("individualPosition"),
                                                  col("participant_info.championId").alias("championId"),
                                                  col("participant_info.championName").alias("championName"),
                                                  col("participant_info.champLevel").alias("champLevel"),
                                                  col("participant_info.kills").alias("kills"),
                                                  col("participant_info.deaths").alias("deaths"),
                                                  col("participant_info.assists").alias("assists"),
                                                  col("participant_info.totalMinionsKilled").alias("totalMinionsKilled"),
                                                  col("participant_info.goldEarned").alias("goldEarned"),
                                                  col("participant_info.goldSpent").alias("goldSpent"),
                                                  col("participant_info.timePlayed").alias("timePlayed"),
                                                  col("participant_info.doubleKills").alias("doubleKills"),
                                                  col("participant_info.tripleKills").alias("tripleKills"),
                                                  col("participant_info.quadraKills").alias("quadraKills"),
                                                  col("participant_info.pentaKills").alias("pentaKills"),
                                                  col("participant_info.killingSprees").alias("killingSprees"),
                                                  col("participant_info.visionScore").alias("visionScore"),
                                                  col("participant_info.dragonKills").alias("dragonKills"),
                                                  col("participant_info.turretKills").alias("turretKills"),
                                                  col("participant_info.inhibitorKills").alias("inhibitorKills"),
                                                  col("participant_info.nexusKills").alias("nexusKills"),
                                                  col("participant_info.inhibitorTakedowns").alias("inhibitorTakedowns"),
                                                  col("participant_info.inhibitorsLost").alias("inhibitorsLost"),
                                                  col("participant_info.objectivesStolen").alias("objectivesStolen"),
                                                  col("participant_info.item0").alias("item0"),
                                                  col("participant_info.item1").alias("item1"),
                                                  col("participant_info.item2").alias("item2"),
                                                  col("participant_info.item3").alias("item3"),
                                                  col("participant_info.item4").alias("item4"),
                                                  col("participant_info.item5").alias("item5"),
                                                  col("participant_info.item6").alias("item6"))
participants_info = participants_info.withColumn("timePlayed", round(col("participant_info.timePlayed") / 60, 2))
participants_info = participants_info.withColumnRenamed("timePlayed", "timePlayed(mins)")
participants_info = participants_info.drop("participant_info")

#EXPLODE INFOTEAMS TO GET BANS
info_teams = df_matches_details_spark.select("infoTeams")
bans = info_teams.withColumn("bans", col("infoTeams.bans").alias("bans"))
bans = bans.withColumn("bans", explode("bans"))
bans = bans.withColumn("banChampionId", col("bans.championId").alias("championId"))
bans = bans.withColumn("banChampionId", explode("banChampionId"))
bans = bans.drop("infoTeams","bans")



#JOIN MATCHES_DETAILS AND BANS
participants = participants.withColumn("id", monotonically_increasing_id())
participants_info = participants_info.withColumn("id", monotonically_increasing_id())
bans = bans.withColumn("id", monotonically_increasing_id())
df_matches_details_spark = df_matches_details_spark.withColumn("id", monotonically_increasing_id())

df_matches_details_spark = df_matches_details_spark.drop("matchId", "participant_id", "participant_info", "infoTeams")

df_matches_details_spark = df_matches_details_spark.join(participants, on="id")\
                                                   .join(participants_info, on="id")\
                                                   .join(bans, on="id")
                                                   
df_matches_details_spark = df_matches_details_spark.withColumnRenamed("participant_id", "participantId")

cols_to_select = [
    'puuid',
    'summonerId',
    'participantId',
    'matchId',
    'teamId',
    'summonerName',
    'summonerLevel',
    'win',
    'lane',
    'role',
    'teamPosition',
    'individualPosition',
    'championId',
    'championName',
    'champLevel',
    'kills',
    'deaths',
    'assists',
    'banChampionId',
    'totalMinionsKilled',
    'goldEarned',
    'goldSpent',
    'timePlayed(mins)',
    'doubleKills',
    'tripleKills',
    'quadraKills',
    'pentaKills',
    'killingSprees',
    'visionScore',
    'dragonKills',
    'turretKills',
    'inhibitorKills',
    'nexusKills',
    'inhibitorTakedowns',
    'inhibitorsLost',
    'objectivesStolen',
    'item0',
    'item1',
    'item2',
    'item3',
    'item4',
    'item5',
    'item6'
]

df_matches_details_spark = df_matches_details_spark.select(cols_to_select)

In [61]:
df_matches_details_spark.show(1, truncate=False)

+------------------------------------------------------------------------------+----------------------------------------------+------------------------------------------------------------------------------+--------------+------+------------+-------------+-----+----+-------+------------+------------------+----------+------------+----------+-----+------+-------+-------------+------------------+----------+---------+----------------+-----------+-----------+-----------+----------+-------------+-----------+-----------+-----------+--------------+----------+------------------+--------------+----------------+-----+-----+-----+-----+-----+-----+-----+
|puuid                                                                         |summonerId                                    |participantId                                                                 |matchId       |teamId|summonerName|summonerLevel|win  |lane|role   |teamPosition|individualPosition|championId|championName|champLevel|kills|dea

ITEMS TRANSFORM

In [62]:
df_items_spark = df_items_spark.select("id", "name", "description", "plaintext", "into", "gold", "stats")

df_items_spark = df_items_spark.select("*",
                                       col("gold.base").alias("goldBase"),
                                       col("gold.total").alias("goldTotal"),
                                       col("gold.sell").alias("goldSell"),
                                       col("stats.FlatArmorMod").alias("statFlatArmorMod"),
                                       col("stats.FlatCritChanceMod").alias("statFlatCritChanceMod"),
                                       col("stats.FlatHPPoolMod").alias("statFlatHPPoolMod"),
                                       col("stats.FlatHPRegenMod").alias("statFlatHPRegenMod"),
                                       col("stats.FlatMPPoolMod").alias("statFlatMPPoolMod"),
                                       col("stats.FlatMagicDamageMod").alias("statFlatMagicDamageMod"),
                                       col("stats.FlatMovementSpeedMod").alias("statFlatMovementSpeedMod"),
                                       col("stats.FlatPhysicalDamageMod").alias("statFlatPhysicalDamageMod"),
                                       col("stats.FlatSpellBlockMod").alias("statFlatSpellBlockMod"),
                                       col("stats.PercentAttackSpeedMod").alias("statPercentAttackSpeedMod"),
                                       col("stats.PercentLifeStealMod").alias("statPercentLifeStealMod"),
                                       col("stats.PercentMovementSpeedMod").alias("statPercentMovementSpeedMod"),
                                       )

df_items_spark = df_items_spark.na.fill(0.0,subset=["statFlatArmorMod",
                                                "statFlatCritChanceMod",
                                                "statFlatHPPoolMod",
                                                "statFlatHPRegenMod",
                                                "statFlatMPPoolMod",
                                                "statFlatMagicDamageMod",
                                                "statFlatMovementSpeedMod",
                                                "statFlatPhysicalDamageMod",
                                                "statFlatSpellBlockMod",
                                                "statPercentAttackSpeedMod",
                                                "statPercentLifeStealMod",
                                                "statPercentMovementSpeedMod"])

df_items_spark = df_items_spark.drop("gold", "stats")

df_items_spark = df_items_spark.withColumn("into", concat_ws(", ", "into"))

df_items_spark = df_items_spark.withColumn('plaintext', when(col('plaintext') == '', "None").otherwise(col('plaintext')))
df_items_spark = df_items_spark.withColumn('into', when(col('into') == '', "None").otherwise(col('into')))

df_items_spark = df_items_spark.withColumn("description", regexp_replace("description", "<[^>]+>", ""))

In [63]:
df_items_spark.show(1,truncate=False)

+----+-----+-------------+-----------------------------+----------------------------------------------------+--------+---------+--------+----------------+---------------------+-----------------+------------------+-----------------+----------------------+------------------------+-------------------------+---------------------+-------------------------+-----------------------+---------------------------+
|id  |name |description  |plaintext                    |into                                                |goldBase|goldTotal|goldSell|statFlatArmorMod|statFlatCritChanceMod|statFlatHPPoolMod|statFlatHPRegenMod|statFlatMPPoolMod|statFlatMagicDamageMod|statFlatMovementSpeedMod|statFlatPhysicalDamageMod|statFlatSpellBlockMod|statPercentAttackSpeedMod|statPercentLifeStealMod|statPercentMovementSpeedMod|
+----+-----+-------------+-----------------------------+----------------------------------------------------+--------+---------+--------+----------------+---------------------+------------

In [59]:
df_summoner_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/summoner.parquet")
df_ranking_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/ranking.parquet")
df_champions_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/champions.parquet")
df_champions_mastery_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/champions_mastery.parquet")
df_matches_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/matches.parquet")
df_matches_details_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/matches_details.parquet")
df_items_spark = spark_session.read.parquet("C:/Users/Sebastian Esnaola/Desktop/LoL_Data/items.parquet")